1- Get wine data

In [ ]:
import numpy as np 
from sklearn.datasets import load_wine 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler 
from sklearn.model_selection import KFold
from datetime import datetime


class WineData:

    #Get data without cross-validation
    def prepareData(self):

        seed = 1234

        np.random.seed(seed)

        x,y = load_wine(return_X_y = True)

        x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.9)

        scaler = MaxAbsScaler()
        x_tr = scaler.fit_transform(x_tr)

        x_test = scaler.transform(x_test)
        x_test = np.clip(x_test,0,1)


        return np, x_tr, x_test, y_tr, y_test
    
    #Get data with cross-validation = 16
    def prepareNFoldData(self):

        seed = 1234

        np.random.seed(seed)

        x,y = load_wine(return_X_y = True)

        scaler = MaxAbsScaler()
        x = scaler.fit_transform(x)
        x = np.clip(x, 0, 1)  # Clip values to [0, 1]

        print(f'wine data, len(x): {len(x)}')

        n_folds = 16

        kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
        
        train_data_list = []
        test_data_list = []        
        for train_index, test_index in kf.split(x):
            
            x_tr, x_test = x[train_index], x[test_index]
            y_tr, y_test = y[train_index], y[test_index]

            train_data_list.append((x_tr, y_tr))
            test_data_list.append((x_test, y_test))

        return np, train_data_list, test_data_list


implement feature map

In [ ]:
import pennylane as qml

class MyFeatureMap:

    amplitudeNQubits = 5

    @qml.qnode(qml.device("lightning.qubit", wires = amplitudeNQubits))
    def __getAmplitudeEmdedding(a, b):
        
        qml.AmplitudeEmbedding(
        a, wires=range(MyFeatureMap.amplitudeNQubits), pad_with=0, normalize=True)

        qml.adjoint(qml.AmplitudeEmbedding(
        b, wires=range(MyFeatureMap.amplitudeNQubits), pad_with=0, normalize=True))

        return qml.probs(wires = range(MyFeatureMap.amplitudeNQubits))


My quantum Kernel

In [ ]:

from sklearn.svm import SVC 

from sklearn.metrics import accuracy_score 

class MyKernel:

    np = {}

    myFeatureMap = MyFeatureMap()

    def getQKernel(self, A, B):
        
        return self.np.array([[MyKernel.myFeatureMap.__getAmplitudeEmdedding(a, b)[0] for b in B] for a in A])


Scale data by 11

In [ ]:
def getListOfFoldData():

        myWineData = WineData()

        np, train_data_list, test_data_list = myWineData.prepareNFoldData()

        return np, train_data_list, test_data_list


np, train_data_list, test_data_list = getListOfFoldData()

for fold_index in range(len(train_data_list)):

        x_tr, y_tr = train_data_list[fold_index]
        x_test, y_test = test_data_list[fold_index]
        
        svmKernel = SVC(kernel = MyKernel().getQKernel)

        svm = svmKernel.fit(x_tr, x_test)

        svmPrediction = svm.predict(x_test)

        myAccuracyScore = accuracy_score(svmPrediction, y_test)

        print(f'myAccuracyScore: {myAccuracyScore} for kfold: {fold_index}')